## Importando librerias

In [1]:
import pandas as pd

In [ ]:
https://prod.liveshare.vsengsaas.visualstudio.com/join?219095ECD35614CA38C022875BE0F34AB753

# Mensaje de prueba escrito desde otro lado

este mesanej fue eho por otra persona

In [2]:
print("hola mundo")

hola mundo


In [3]:
print("hola")

hola


In [4]:
print("hola")

hola
